In [9]:
!pip install transformers datasets peft accelerate evaluate scikit-learn --quiet


In [10]:
import pandas as pd
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from peft import LoraConfig, get_peft_model
import evaluate
import joblib


In [12]:
def load_data(path):
    try:
        df = pd.read_csv(path)
        if df.shape[1] == 1:
            df = pd.read_csv(path, header=None, names=['text', 'label'])
    except:
        df = pd.read_csv(path, header=None, names=['text', 'label'])
    return df

train_df = load_data('/content/train.csv')
val_df = load_data('/content/val.csv')
test_df = load_data('/content/test.csv')

print("✅ Train sample:")
print(train_df.head())


✅ Train sample:
                                                text    label
0                            i didnt feel humiliated  sadness
1  i can go from feeling so hopeless to so damned...  sadness
2   im grabbing a minute to post i feel greedy wrong    anger
3  i am ever feeling nostalgic about the fireplac...     love
4                               i am feeling grouchy    anger


In [ ]:
le = LabelEncoder()
train_df['label'] = le.fit_transform(train_df['label'])
val_df['label'] = le.transform(val_df['label'])
test_df['label'] = le.transform(test_df['label'])

num_labels = len(le.classes_)
print("🎯 Labels:", list(le.classes_))


🎯 Labels: ['anger', 'fear', 'joy', 'love', 'sadness', 'surprise']


In [14]:
train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)
test_ds = Dataset.from_pandas(test_df)


In [15]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

train_ds = train_ds.map(tokenize_function, batched=True)
val_ds = val_ds.map(tokenize_function, batched=True)
test_ds = test_ds.map(tokenize_function, batched=True)

train_ds = train_ds.rename_column("label", "labels")
val_ds = val_ds.rename_column("label", "labels")
test_ds = test_ds.rename_column("label", "labels")

train_ds.set_format("torch")
val_ds.set_format("torch")
test_ds.set_format("torch")


Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [16]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_lin", "v_lin"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 742,662 || all params: 67,700,748 || trainable%: 1.0970


In [17]:
data_collator = DataCollatorWithPadding(tokenizer)
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return metric.compute(predictions=preds, references=labels)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    report_to="none"   # 🚫 disables wandb and all external loggers
)


In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()


/tmp/ipython-input-3198824983.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.383900,0.250948,0.908000
2,0.251700,0.201229,0.925500
3,0.185800,0.183582,0.924000


TrainOutput(global_step=3000, training_loss=0.34089174143473305, metrics={'train_runtime': 374.2597, 'train_samples_per_second': 128.253, 'train_steps_per_second': 8.016, 'total_flos': 1617099669504000.0, 'train_loss': 0.34089174143473305, 'epoch': 3.0})

In [19]:
results = trainer.evaluate(test_ds)
print(f"✅ Test Accuracy: {results['eval_accuracy']:.4f}")


✅ Test Accuracy: 0.9200


In [20]:
save_path = "/content/fine_tuned_lora_emotion_model"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
joblib.dump(le, f"{save_path}/label_encoder.pkl")

print("💾 Model saved at:", save_path)


💾 Model saved at: /content/fine_tuned_lora_emotion_model


In [21]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from peft import PeftModel
import torch
import joblib

# ===== Paths =====
base_model_name = "distilbert-base-uncased"  # same as you used in training
save_path = "/content/fine_tuned_lora_emotion_model"  # your LoRA output folder

# ===== Load tokenizer =====
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# ===== Load base model + LoRA adapters =====
base_model = AutoModelForSequenceClassification.from_pretrained(
    base_model_name,
    num_labels=6,  # change to your number of emotion labels
)
model = PeftModel.from_pretrained(base_model, save_path)

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ===== Load label encoder =====
le = joblib.load(f"{save_path}/label_encoder.pkl")

# ===== Build pipeline =====
classifier = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1,
)

# ===== Prediction function =====
def predict_emotion(text):
    result = classifier(text, truncation=True, max_length=128)[0]
    if "LABEL_" in result["label"]:
        label_id = int(result["label"].split("_")[-1])
        emotion = le.inverse_transform([label_id])[0]
    else:
        emotion = result["label"]
    print(f"🗣️ Text: {text}\n💬 Predicted Emotion: {emotion}\n🔹 Confidence: {result['score']:.3f}")

# ===== Example =====
predict_emotion("I want ")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


🗣️ Text: I want 
💬 Predicted Emotion: love
🔹 Confidence: 0.389


In [22]:
# ⚡ Emotion classifier + TinyLlama response (optimized for Colab)

!pip install transformers peft torch accelerate joblib --quiet

from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, AutoModelForCausalLM
from peft import PeftModel
import torch, joblib, gc

# ===== Load emotion classifier =====
base_model_name = "distilbert-base-uncased"
save_path = "/content/fine_tuned_lora_emotion_model"

tokenizer_cls = AutoTokenizer.from_pretrained(base_model_name)
base_model = AutoModelForSequenceClassification.from_pretrained(base_model_name, num_labels=6)
model_cls = PeftModel.from_pretrained(base_model, save_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_cls.to(device)
le = joblib.load(f"{save_path}/label_encoder.pkl")

classifier = pipeline(
    "text-classification",
    model=model_cls,
    tokenizer=tokenizer_cls,
    device=0 if torch.cuda.is_available() else -1,
)

# ===== Free unnecessary GPU memory =====
gc.collect(); torch.cuda.empty_cache()

# ===== Load tiny LLM for responses =====
llm_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
llm_tokenizer = AutoTokenizer.from_pretrained(llm_name)
llm_model = AutoModelForCausalLM.from_pretrained(
    llm_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    low_cpu_mem_usage=True
).to(device)



Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [24]:
# ===== Function for emotion-based LLM response =====
def emotion_based_reply(user_text):
    result = classifier(user_text, truncation=True, max_length=128)[0]
    if "LABEL_" in result["label"]:
        label_id = int(result["label"].split("_")[-1])
        emotion = le.inverse_transform([label_id])[0]
    else:
        emotion = result["label"]
    confidence = result["score"] * 100

    prompt = (
        f"User said: '{user_text}'.\n"
        f"The detected emotion is {emotion}.\n"
        f"Respond empathetically in one or two sentences."
    )

    inputs = llm_tokenizer(prompt, return_tensors="pt").to(device)
    outputs = llm_model.generate(**inputs, max_new_tokens=80, temperature=0.8, top_p=0.9, do_sample=True)
    reply = llm_tokenizer.decode(outputs[0], skip_special_tokens=True).split(prompt)[-1].strip()

    print(f"🧩 Text: {user_text}")
    print(f"💬 Emotion: {emotion.upper()} ({confidence:.1f}%)")
    print(f"🤖 Response: {reply}\n")

# ===== Examples =====
emotion_based_reply("I'm so happy ")
# emotion_based_reply("I'm so happy today!")
# emotion_based_reply("I'm worried about the future.")


🧩 Text: I'm so happy 
💬 Emotion: JOY (99.1%)
🤖 Response: The given text contains emotions expressed in a positive way.
You can also check if the text contains positive or negative emotions using the Emotion Analyzer app.
The text contains negative emotions.

